In [1]:
from operator import itemgetter

import opencc
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_together.llms import Together

load_dotenv()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_prompt = """
"""

user_prompt = """
{question}
"""


def s2hk(content):
    converter = opencc.OpenCC("s2hk")
    return converter.convert(content)


def get_answer(question, system_prompt, user_prompt):
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template(user_prompt),
        ]
    )

    # model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
    model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)
    # model = AzureChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
    # model = Together(model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.7, max_tokens=4096)
    # model = Together(model="deepseek-ai/deepseek-coder-33b-instruct", temperature=0.7, max_tokens=4096)

    chain = (
        RunnablePassthrough.assign(chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"))
        | prompt
        | model
        | StrOutputParser()
        | RunnableLambda(s2hk)
    )

    with get_openai_callback() as callback:
        response = chain.invoke({"question": question})
        print(callback, end="\n\n")
    print(response)
    memory.save_context({"question": question}, {"response": response})

In [2]:
get_answer("Tell me some useful info and tips about AI.", system_prompt, user_prompt)

Tokens Used: 626
	Prompt Tokens: 132
	Completion Tokens: 494
Successful Requests: 1
Total Cost (USD): $0.001186

Of course! Here are some useful information and tips about AI:

1. Understanding AI:
   - Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning (acquiring information and rules for using the information), reasoning (using rules to reach approximate or definite conclusions), and self-correction.

2. Types of AI:
   - AI can be categorized into Narrow AI (also known as Weak AI) and General AI (also known as Strong AI).
   - Narrow AI is designed for a specific task, such as virtual assistants like Siri or Alexa. These systems are good at one task but cannot perform others.
   - General AI would have the ability to understand, learn, and apply knowledge across various tasks, similar to human intelligence. However, this level of AI does not currently exist.

3. Machine Lear